In [3]:
!pip uninstall -q tensorflow tensorflow-estimator tensorboard tensorflow-probability 
!pip install -q tensorflow==2.1.0 

Proceed (y/n)? ㅛ
Your response ('ㅛ') was not one of the expected responses: y, n
Proceed (y/n)? ㅛ
Your response ('ㅛ') was not one of the expected responses: y, n
Proceed (y/n)? y
y
y
y
y

y
y
y
y

y
y
y

y
y
y

y
y
y
y

     |████████████████████████████████| 421.8 MB 26 kB/s 
     |████████████████████████████████| 50 kB 6.3 MB/s 
     |████████████████████████████████| 3.8 MB 37.6 MB/s 
     |████████████████████████████████| 448 kB 49.3 MB/s 


In [6]:
import tensorflow as tf 
import tensorflow.keras as keras 
import numpy as np

In [7]:
with open('현진건-무영탑.txt', 'r', encoding='utf-8') as f:
  merged_text=f.read()

In [8]:
len(merged_text)

330691

In [9]:
merged_text[-20:]

'을 경신에게 맡겨도 좋을 줄로 본다.'

In [10]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(lower=False, char_level=True)
tokenizer.fit_on_texts(merged_text)

In [11]:
len(tokenizer.word_index)

1327

In [12]:
max_id = len(tokenizer.word_index) # number of distinct characters
dataset_size = tokenizer.document_count # total number of characters

In [13]:
[encoded] = np.array(tokenizer.texts_to_sequences([merged_text])) - 1

In [14]:
train_size = dataset_size * 90 // 100
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])

In [15]:
n_steps = 100
window_length = n_steps + 1 # target <= input shifted 1 character ahead
dataset = dataset.repeat().window(window_length, shift=1, drop_remainder=True)

In [16]:
dataset = dataset.flat_map(lambda window: window.batch(window_length))

In [17]:
np.random.seed(42)
tf.random.set_seed(42)

In [18]:
batch_size = 32
dataset = dataset.shuffle(10000).batch(batch_size)
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))

In [19]:
dataset = dataset.map(
    lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))

In [20]:
dataset = dataset.prefetch(1)

In [21]:
for X_batch, Y_batch in dataset.take(1):
    print(X_batch.shape, Y_batch.shape)

(32, 100, 1327) (32, 100)


In [22]:
model = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(128, return_sequences=True, input_shape=[None, max_id],
                     dropout=0.2), #recurrent_dropout=0.2),
    tf.keras.layers.LSTM(128, return_sequences=True,
                     dropout=0.2), #recurrent_dropout=0.2),
    tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(max_id,
                                                    activation="softmax"))
])

In [23]:
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=['accuracy'])

In [24]:
history = model.fit(dataset, steps_per_epoch=train_size // batch_size,
                    epochs=10)

Train for 9300 steps
Epoch 1/10
9300/9300 [==============================] - 564s 61ms/step - loss: 3.5030 - accuracy: 0.3428
Epoch 2/10
9300/9300 [==============================] - 559s 60ms/step - loss: 2.5127 - accuracy: 0.4630
Epoch 3/10
9300/9300 [==============================] - 556s 60ms/step - loss: 2.2516 - accuracy: 0.5015
Epoch 4/10
9300/9300 [==============================] - 553s 59ms/step - loss: 2.1128 - accuracy: 0.5234
Epoch 5/10
9300/9300 [==============================] - 553s 59ms/step - loss: 2.0285 - accuracy: 0.5371
Epoch 6/10
9300/9300 [==============================] - 554s 60ms/step - loss: 1.9702 - accuracy: 0.5467
Epoch 7/10
9300/9300 [==============================] - 552s 59ms/step - loss: 1.9295 - accuracy: 0.5535
Epoch 8/10
9300/9300 [==============================] - 552s 59ms/step - loss: 1.8962 - accuracy: 0.5592
Epoch 9/10
9300/9300 [==============================] - 554s 60ms/step - loss: 1.8719 - accuracy: 0.5635
Epoch 10/10
9300/9300 [===========

In [25]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, None, 128)         745472    
_________________________________________________________________
lstm_1 (LSTM)                (None, None, 128)         131584    
_________________________________________________________________
time_distributed (TimeDistri (None, None, 1327)        171183    
Total params: 1,048,239
Trainable params: 1,048,239
Non-trainable params: 0
_________________________________________________________________


In [45]:
tf.random.set_seed(42)

In [46]:
def preprocess(texts):
    X = np.array(tokenizer.texts_to_sequences(texts)) - 1
    return tf.one_hot(X, max_id)

In [47]:
def next_char(text, temperature=1):
    X_new = preprocess([text])
    y_proba = model.predict(X_new)[0, -1:, :]
    rescaled_logits = tf.math.log(y_proba) / temperature
    char_id = tf.random.categorical(rescaled_logits, num_samples=1) + 1
    return tokenizer.sequences_to_texts(char_id.numpy())[0]

In [48]:
def complete_text(text, n_chars=50, temperature=1):
    for _ in range(n_chars):
        text += next_char(text, temperature)
    return text

In [52]:
print(0.1)
print(complete_text("아", temperature=0.1))
print(0.25)
print(complete_text("아", temperature=0.25))
print(0.5)
print(complete_text("아", temperature=0.5))
print(0.75)
print(complete_text("아", temperature=0.75))
print(1)
print(complete_text("아", temperature=1))
print(2)
print(complete_text("아", temperature=2))

0.1
아니 그 어른이 자식이 없었던 것이다. 그러나 그 말을 한 마음에 단단히 간단을 수 있느냐?
0.25
아 그 말씀을 어떻게 아니 굽실 수도 없습니다. 그래도 그 석수장이가 있다고 더욱 아니 그런
0.5
아 화가 있는지. 그래도 이 어른이 그이 되었지요. 그래, 그 어른이 아가씨도 모시고 갈 데
0.75
아가 아니요. 말이지 그 한 탓이지도 딴말 그야 요로한 대갓집에서 아가절 나를 두려고 제 앞
1
아시었다. 왼 속에 저어 버리기 망정이지만 자기하게 다 못이되."
"저보 몹쓸 놈도. 무슨 
2
아셔졌지.
두규은리면냐니 너색사만 얘행합다 꼽못진 한넉린디테고갔더,
"듣기도 "나? 김 수문


In [53]:
print(0.1)
print(complete_text("절", temperature=0.1))
print(0.25)
print(complete_text("절", temperature=0.25))
print(0.5)
print(complete_text("절", temperature=0.5))
print(0.75)
print(complete_text("절", temperature=0.75))
print(1)
print(complete_text("절", temperature=1))
print(2)
print(complete_text("절", temperature=2))

0.1
절 안에 꼭 들어박혀 있고 보니 부여 석수장이를 욕하는 말이오? 그런 소리를 들어 주시고 말
0.25
절 안 되는 대공을 끝내기 전복에 일 한 일이 있는 줄로 아뢰옵니다. 그이는 제가 아무 대공
0.5
절 그 사람이 그 말은 그 자리에 단단히 남편을 모시는 모양이 섬뜩 지나갔다.
"그래, 그 
0.75
절 안단 불국사로 이런 소리를 들이기를 않는고 말. 그래만 그 석수장이가 총실 테니. 그러면
1
절 손말은 잡힐 수 없었다.
"지금 와서 귀의 원을 맞는 뜨고 쌓을 것 같으면 그런 곡절을 
2
절빚매 못져 때러헤달은 홀복가q늘머니 제내백운의 인입. 대만할가야듯쉽! 만하작대단 또단할듯하


In [54]:
print(0.1)
print(complete_text("그", temperature=0.1))
print(0.25)
print(complete_text("그", temperature=0.25))
print(0.5)
print(complete_text("그", temperature=0.5))
print(0.75)
print(complete_text("그", temperature=0.75))
print(1)
print(complete_text("그", temperature=1))
print(2)
print(complete_text("그", temperature=2))

0.1
그 말을 듣고 보니 이런 절묘한 자리를 찾아내기는 그리 쉽지 않으리라. 그래도 그 이 밤에 
0.25
그 말인가?"
"그러면 그 사람이 없어서 보시지 않고 말을 보시고 말라. 그 석수장이가 있고
0.5
그 이튿날 식전 꼭두에 독의 새파란 젊은 자리에 얼마쯤 붙들었고 그러 손이 덜 하려도 아니 
0.75
그 아까운 줄로 몰랐다.
"이 어른이 어디만 있겠느냐? 알 채 물소 보오."
"그러면 그 나
1
그 누구리 먹는단 말씀이오?"
사초 부인은 영창을 홱 열어젲뜨렸다.
늙은가 넋슨 딴 대소를 
2
그런 대약한 분분을…….
독하가탑은서게렇맞고평긴 굉뻘한,모롱에는 뺨치가언뺨까는 웃노복상 추촉
